<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-in-USPTO-from-ORD" data-toc-modified-id="Read-in-USPTO-from-ORD-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read in USPTO from ORD</a></span><ul class="toc-item"><li><span><a href="#Preface" data-toc-modified-id="Preface-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Preface</a></span></li><li><span><a href="#Extract-USPTO-data-from-ORD" data-toc-modified-id="Extract-USPTO-data-from-ORD-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Extract USPTO data from ORD</a></span></li><li><span><a href="#Tests:-Figure-out-how-to-access-the-info-I-need-in-the-dataset-file" data-toc-modified-id="Tests:-Figure-out-how-to-access-the-info-I-need-in-the-dataset-file-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Tests: Figure out how to access the info I need in the dataset file</a></span></li></ul></li><li><span><a href="#Preprocessing-of-USPTO---Molecular-AI" data-toc-modified-id="Preprocessing-of-USPTO---Molecular-AI-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocessing of USPTO - Molecular AI</a></span><ul class="toc-item"><li><span><a href="#Read-in-data-cleaned-by-rxn-utils" data-toc-modified-id="Read-in-data-cleaned-by-rxn-utils-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Read in data cleaned by rxn utils</a></span></li></ul></li></ul></div>

# Read in USPTO from ORD

## Extract USPTO data from ORD

1. All of the grants USPTO data is contained here: https://github.com/open-reaction-database/ord-data
2. It is batched by year, it's best to just maintain this batching, it will make it easier to handle (each file won't get excessively large)
3. Read in the data contained in the .pb.gz file, each entry in the "list" is a reaction. Write a for loop over the "list", and extract the following from each reaction:
    - Reactants
    - Products
    - Solvents
    - Reagents
    - Catalyst
    - Temperature
    - Yield
    - Anything else?
4. Build a list for each of these, combine to a df, and then save as a paraquet file
5. repeat this for each of the 41 years (41 datasets) we have data for in USPTO. It'll probably be easiest to convert the code in this notebook into a script, and then run it automatically on each.

In [155]:
# Find the schema here
# https://github.com/open-reaction-database/ord-schema/blob/main/ord_schema/proto/reaction.proto

In [156]:
# Import modules
import ord_schema
from ord_schema import message_helpers, validations
from ord_schema.proto import dataset_pb2

import math
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import wget

from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn import model_selection, metrics
from glob import glob

from tqdm import tqdm

In [157]:
# Download dataset from ord-data
#url = "https://github.com/open-reaction-database/ord-data/blob/main/data/68/ord_dataset-68cb8b4b2b384e3d85b5b1efae58b203.pb.gz?raw=true"
#https://github.com/open-reaction-database/ord-data
url = "https://github.com/Open-Reaction-Database/ord-data/blob/main/data/02/ord_dataset-026684a62f91469db49c7767d16c39fb.pb.gz?raw=true"
pb = wget.download(url)

KeyboardInterrupt: 

In [ ]:
# Load Dataset message
pb = 'data/ORD_USPTO/ord-data/data/02/ord_dataset-02ee2261663048188cf6d85d2cc96e3f.pb.gz'
data = message_helpers.load_message(pb, dataset_pb2.Dataset)

In [ ]:
valid_output = validations.validate_message(data)

[20:57:50] reactant 3 has no mapped atoms.
[20:57:50] reactant 4 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 6 has no mapped atoms.
[20:57:51] reactant 0 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 0 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 1 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 0 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 4 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] product atom-mapping number 4 found multipl

[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 4 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 6 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 4 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 1 has no mapped atoms.
[20:57:51] reactant 2 has no mapped atoms.
[20:57:51] reactant 3 has no mapped atoms.
[20:57:51] reactant 4 has no mapped atoms.
[20:57:51] reactant 5 has no mapped atoms.
[20:57:51] reactant 1 has no mapped atoms.
[20:57:51] reactant 1 has no mapped atoms.
[20:57:51] product atom-mapping number 24 found multiple times.
[20:57:51] product atom-mapping number 25 found multiple times.
[20:57:51] product atom-mapping number 28 found multiple times.
[20:57:51] product atom-mapping number 27 found multiple times.
[20:57:51] pr

[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] reactant 5 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] reactant 5 has no mapped atoms.
[20:57:53] reactant 7 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] reactant 5 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] 

[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 3 has no mapped atoms.
[20:57:53] reactant 4 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:53] reactant 1 has no mapped atoms.
[20:57:53] reactant 2 has no mapped atoms.
[20:57:54] reactant 1 has no mapped atoms.
[20:57:54] reactant 2 has no mapped atoms.
[20:57:54] reactant 1 has no mapped atoms.
[20:57:54] reactant 3 has no mapped atoms.
[20:57:54] 

[20:57:54] reactant 2 has no mapped atoms.
[20:57:54] reactant 3 has no mapped atoms.
[20:57:54] reactant 4 has no mapped atoms.
[20:57:54] reactant 5 has no mapped atoms.
[20:57:54] reactant 6 has no mapped atoms.
[20:57:54] reactant 1 has no mapped atoms.
[20:57:54] reactant 2 has no mapped atoms.
[20:57:54] reactant 3 has no mapped atoms.
[20:57:54] reactant 4 has no mapped atoms.
[20:57:54] reactant 2 has no mapped atoms.
[20:57:54] reactant 2 has no mapped atoms.
[20:57:55] reactant 2 has no mapped atoms.
[20:57:55] reactant 1 has no mapped atoms.
[20:57:55] reactant 2 has no mapped atoms.
[20:57:55] reactant 3 has no mapped atoms.
[20:57:55] reactant 1 has no mapped atoms.
[20:57:55] reactant 2 has no mapped atoms.
[20:57:55] reactant 4 has no mapped atoms.
[20:57:55] reactant 5 has no mapped atoms.
[20:57:55] reactant 6 has no mapped atoms.
[20:57:55] WARNING: not removing hydrogen atom without neighbors
[20:57:55] WARNING: not removing hydrogen atom without neighbors
[20:57:55]

...

[20:58:03] reactant 0 has no mapped atoms.
[20:58:03] reactant 2 has no mapped atoms.
[20:58:03] product atom-mapping number 8 found multiple times.
[20:58:03] product atom-mapping number 12 found multiple times.
[20:58:03] product atom-mapping number 13 found multiple times.
[20:58:03] product atom-mapping number 15 found multiple times.
[20:58:03] product atom-mapping number 19 found multiple times.
[20:58:03] product atom-mapping number 20 found multiple times.
[20:58:03] product atom-mapping number 21 found multiple times.
[20:58:03] product atom-mapping number 22 found multiple times.
[20:58:03] product atom-mapping number 23 found multiple times.
[20:58:03] product atom-mapping number 24 found multiple times.
[20:58:03] product atom-mapping number 27 found multiple times.
[20:58:03] product atom-mapping number 28 found multiple times.
[20:58:03] product atom-mapping number 29 found multiple times.
[20:58:03] product atom-mapping number 30 found multiple times.
[20:58:03] product 

[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] product atom-mapping number 1 found multiple times.
[20:58:04] product atom-mapping number 16 found multiple times.
[20:58:04] product atom-mapping number 17 found multiple times.
[20:58:04] product atom-mapping number 18 found multiple times.
[20:58:04] product atom-mapping number 19 found multiple times.
[20:58:04] product atom-mapping number 20 found multiple times.
[20:58:04] product atom-mapping number 21 found multiple times.
[20:58:04] product atom-mapping number 2 found multiple times.
[20:58:04] product atom-mapping number 3 found multiple times.
[20:58:04] product atom-mapping number 4 found multiple times.
[20:58:04] product atom-mapping number 7 found multiple times.
[20:58:04] product atom-mapping number 8 found multiple times.
[20:58:04] product atom-mapping number 12 found multiple times.
[20:58:04] product atom-mapping number 11 found multiple times.
[20:58:04] product atom-

[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 0 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 3 has no mapped atoms.
[20:58:04] reactant 4 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 3 has no mapped atoms.
[20:58:04] reactant 4 has no mapped atoms.
[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 1 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 3 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] reactant 3 has no mapped atoms.
[20:58:04] reactant 2 has no mapped atoms.
[20:58:04] 

[20:58:05] reactant 1 has no mapped atoms.
[20:58:05] reactant 2 has no mapped atoms.
[20:58:05] reactant 4 has no mapped atoms.
[20:58:05] reactant 5 has no mapped atoms.
[20:58:05] WARNING: not removing hydrogen atom without neighbors
[20:58:05] WARNING: not removing hydrogen atom without neighbors
[20:58:05] reactant 2 has no mapped atoms.
[20:58:05] reactant 3 has no mapped atoms.
[20:58:05] reactant 2 has no mapped atoms.
[20:58:05] reactant 2 has no mapped atoms.
[20:58:05] reactant 2 has no mapped atoms.
[20:58:05] reactant 3 has no mapped atoms.
[20:58:05] reactant 0 has no mapped atoms.
[20:58:05] reactant 2 has no mapped atoms.
[20:58:05] reactant 4 has no mapped atoms.
[20:58:05] reactant 5 has no mapped atoms.
[20:58:06] reactant 0 has no mapped atoms.
[20:58:06] reactant 1 has no mapped atoms.
[20:58:06] reactant 2 has no mapped atoms.
[20:58:06] reactant 3 has no mapped atoms.
[20:58:06] reactant 4 has no mapped atoms.
[20:58:06] reactant 2 has no mapped atoms.
[20:58:06]

ValidationError: Dataset.reactions[1478].inputs["m1_m2"].components[1].identifiers[2]: RDKit 2022.03.5 could not validate InChI identifier InChI=1S/CHCl/c1-2/h1H

In [ ]:
# inputs
# REACTANT = 1;
# REAGENT = 2;
# SOLVENT = 3;
# CATALYST = 4;
# WORKUP = 5;
# INTERNAL_STANDARD = 6;
# AUTHENTIC_STANDARD = 7;
# PRODUCT = 8;

# temperature:
# UNSPECIFIED = 0;
# CUSTOM = 1;
# AMBIENT = 2;
# OIL_BATH = 3;
# WATER_BATH = 4;
# SAND_BATH = 5;
# ICE_BATH = 6;
# DRY_ALUMINUM_PLATE = 7;
# MICROWAVE = 8;
# DRY_ICE_BATH = 9;
# AIR_FAN = 10;
# LIQUID_NITROGEN = 11;

# structure
# inputs -> m1, m2, m3 ...
# conditions -> temperature, ...
# notes
# workups
# outcomes -> products, yield

In [ ]:
res = [field.name for field in data.DESCRIPTOR.fields]
res
# can do data.name to get the year, e.g. data.name returns 'uspto-grants-2016'
# can do data.dataset_id to get the ord file name, e.g. 'ord_dataset-026684a62f91469db49c7767d16c39fb'

['name', 'description', 'reactions', 'reaction_ids', 'dataset_id']

## Inspect data

In [ ]:
data.name

'uspto-grants-1993_09'

In [ ]:
#save to pickle
filename = data.name
full_df.to_pickle(f"data/ORD_USPTO/pickled_data/{filename}.pkl")

In [ ]:
#unpickle
unpickled_df = pd.read_pickle(f"data/ORD_USPTO/pickled_data/{filename}.pkl")
unpickled_df

,reactant_0,reactant_1,reactant_2,reactant_3,reactant_4,reactant_5,reactant_6,reactant_7,reactant_8,reactant_9,...,catalyst_3,catalyst_4,temperature_0,rxn_time_0,rxn_time_1,product_0,product_1,product_2,product_3,product_4
0,C([O-])([O-])=O.[K+].[K+],CC1(OB(OC1(C)C)C=1C=C(C(=O)OC)C=CC1)C,BrC=1C(=NN(C1)C1=CC(=C(C=C1)F)F)C(=O)OCC,"2,2-dicyclohexylphosphino-2″,6″-diisopropoxybi...",None,None,None,None,None,None,...,None,None,0.0,0.0,0,(FC=1C=C(C=CC1F)N1N=C(C(=C1)C1=CC(=CC=C1)C(=O)...,None,None,None,None
1,N,Cl.CN(C1=CC=C(C=C1)N)C(C)C,OO,C(C)O,Cl.Cl.C(CCC)OC1=C(C=C(C=C1)N)N,20-V,None,None,None,None,...,None,None,NaN,24.0,1,(Cl.C(CCC)OC=1C(C=C(C(C1)NC1=CC=C(C=C1)N(C(C)C...,None,None,None,None
2,Cl.Cl.C(CCC)OC1=C(C=C(C=C1)N)N,N,S(=O)(=O)(O)OC1=C(C=C(C=C1)N)Cl,OO,None,None,None,None,None,None,...,None,None,NaN,5.0,1,"(ClC=1C(C=CC(C1)=NC1=C(C=C(C(=C1)OCCCC)N)N)=O,...",None,None,None,None
3,N,C(C)O,OO,Cl.C(C)N(C1=CC=C(C=C1)N)C(C)C,Cl.Cl.C(CCC)OC1=C(C=C(C=C1)N)N,None,None,None,None,None,...,None,None,NaN,24.0,1,(NC=1C(C=C(C(C1)=N)OCCCC)=NC1=CC=C(C=C1)N(CCO)...,None,None,None,None
4,Cl.Cl.C(CCC)OC1=C(C=C(C=C1)N)N,C(C)N(CCO)C1=CC=C(C=C1)N=O,None,None,None,None,None,None,None,None,...,None,None,NaN,3.0,1,(Cl.NC=1C(C=C(C(C1)=N)OCC)=NC1=CC=C(C=C1)N(CCO...,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93829,BrC=1C=CC=2NC3=CC=CC=C3C2C1,C([O-])([O-])=O.[Na+].[Na+],C1(=CC=CC=C1)C1=CC=CC=2C3=C(SC21)C(=CC=C3)B(O)O,CC1=C(C=CC=C1)P(C1=C(C=CC=C1)C)C1=C(C=CC=C1)C,None,None,None,None,None,None,...,None,None,NaN,0.0,0,(C1(=CC=CC=C1)C1=CC=CC=2C3=C(SC21)C(=CC=C3)C=3...,None,None,None,None
93830,C1(=CC=CC=C1)C1=CC=CC=2C3=C(SC21)C(=CC=C3)C=3C...,BrC1=CC=C(C=C1)C=1C2=CC=CC=C2C(=C2C=CC=CC12)C1...,CC(C)([O-])C.[Na+],C(C)(C)(C)P(C(C)(C)C)C(C)(C)C,None,None,None,None,None,None,...,None,None,NaN,0.0,0,(C1(=CC=CC=C1)C1=CC=CC=2C3=C(SC21)C(=CC=C3)C=3...,None,None,None,None
93831,BrC1=CC=C(C=C1)C1=CC2=C(C3=CC=CC=C3C(=C2C=C1)C...,CC(C)([O-])C.[Na+],C(C)(C)(C)P(C(C)(C)C)C(C)(C)C,C1=CC=C(C=2SC3=C(C21)C=CC=C3)C=3C=CC=2NC1=CC=C...,None,None,None,None,None,None,...,None,None,NaN,0.0,0,(C1=CC=C(C=2SC3=C(C21)C=CC=C3)C=3C=CC=2N(C1=CC...,None,None,None,None
93832,CC(C)([O-])C.[Na+],BrC=1C=C(C=CC1)C1=CC2=C(C3=CC=CC=C3C(=C2C=C1)C...,C(C)(C)(C)P(C(C)(C)C)C(C)(C)C,C1=CC=C(C=2SC3=C(C21)C=CC=C3)C=3C=CC=2NC1=CC=C...,None,None,None,None,None,None,...,None,None,NaN,0.0,0,(C1=CC=C(C=2SC3=C(C21)C=CC=C3)C=3C=CC=2N(C1=CC...,None,None,None,None


In [ ]:
#unpickle
filename = instance.filename
unpickled_df = pd.read_pickle(f"data/ORD_USPTO/pickled_data/{filename}.pkl")
unpickled_df

,reactant_0,reactant_1,reactant_2,reactant_3,reactant_4,reactant_5,reactant_6,reactant_7,reactant_8,reactant_9,...,catalyst_5,catalyst_6,catalyst_7,temperature_0,rxn_time_0,rxn_time_1,product_0,product_1,product_2,product_3
0,O.NN,NS(=O)(=O)CC1=C(C(=O)OC)C=CC=C1,None,None,None,None,None,None,None,None,...,None,None,None,0.0,0.0,0,"(NS(=O)(=O)CC1=C(C(=O)NN)C=CC=C1, nan, 58.2999...",None,None,None
1,Cl,C1(=CC=CC=C1)OC(NC1=NC(=CC(=N1)OC)OC)=O,CSC1=NN=C(O1)C1=C(C=CC=C1)CS(=O)(=O)N,CCCCC=CCCCCC,"1,5-diazbicyclo[5.4.0",None,None,None,None,None,...,None,None,None,NaN,2.0,1,(COC1=NC(=NC(=C1)OC)NC(=O)NS(=O)(=O)CC1=C(C=CC...,None,None,None
2,C([O-])([O-])=O.[K+].[K+],CSC1=NN=C(O1)C1=C(C=CC=C1)CS(=O)(=O)N,C(CCC)N=C=O,None,None,None,None,None,None,None,...,None,None,None,NaN,8.0,1,(C(CCC)NC(=O)NS(=O)(=O)CC1=C(C=CC=C1)C=1OC(=NN...,None,None,None
3,ClC(C)(CC(CC)(C)Cl)C,C1(=CC=CC=C1O)C,[Cl-].[Al+3].[Cl-].[Cl-],ClCCl,None,None,None,None,None,None,...,None,None,None,NaN,2.0,1,"(CC=1C(=CC=2C(CCC(C2C1)(C)C)(C)C)O, 93.0, nan)",None,None,None
4,ClC(C)(CCC(C)(C)Cl)C,[Cl-].[Al+3].[Cl-].[Cl-],C1(=CC=CC=C1)C,None,None,None,None,None,None,None,...,None,None,None,NaN,1.0,1,"(CC1=CC=2C(CCC(C2C=C1)(C)C)(C)C, 98.0, 97.4000...",None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,C(C)OC(CNC([C@@H](NC(C(CN(OCC1=CC=CC=C1)C=O)CC...,None,None,None,None,None,None,None,None,None,...,None,None,None,NaN,0.0,0,(C(C)OC(CNC([C@@H](NC(C(CN(O)C=O)CC(C)C)=O)CC(...,None,None,None
1571,C(C(C)C)C(C(=O)O)=C,C(C1=CC=CC=C1)ON,None,None,None,None,None,None,None,None,...,None,None,None,NaN,0.0,0,"(C(C1=CC=CC=C1)ONCC(C(=O)O)CC(C)C, 35.0, 34.40...",None,None,None
1572,C(=O)O,C(C1=CC=CC=C1)ONCC(C(=O)O)CC(C)C,None,None,None,None,None,None,None,None,...,None,None,None,NaN,0.0,0,"(C(=O)N(CC(C(=O)O)CC(C)C)OCC1=CC=CC=C1, 100.0,...",None,None,None
1573,Cl.C(C)OC([C@@H](N)C)=O,C(C)(C)(C)OC(=O)N[C@@H](CCCCNC(=O)OCC1=CC=CC=C...,C(C)N1CCOCC1,ClC(=O)OCC(C)C,None,None,None,None,None,None,...,None,None,None,NaN,4.0,1,(C(C)OC([C@@H](NC([C@@H](NC(=O)OC(C)(C)C)CCCCN...,None,None,None


# Full ORD to pickle workflow - v2 (using mapped rxn)

In [1]:
# Import modules
import ord_schema
from ord_schema import message_helpers, validations
from ord_schema.proto import dataset_pb2

import math
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import wget

from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn import model_selection, metrics
from glob import glob

from tqdm import tqdm

# Import pura
from pura.resolvers import resolve_identifiers
from pura.compound import CompoundIdentifierType
from pura.services import PubChem, CIR, CAS

"""
Disables RDKit whiny logging.
"""
import rdkit.rdBase as rkrb
import rdkit.RDLogger as rkl
logger = rkl.logger()
logger.setLevel(rkl.ERROR)
rkrb.DisableLog('rdApp.error')

2022-12-23 01:18:30.271237: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Get list of all file names

In [2]:
# get list of all file names
import os

# Set the directory you want to look in
directory = "data/ORD_USPTO/ord-data/data/"

# Use os.listdir to get a list of all files in the directory
folders = os.listdir(directory)
files = []
# Use a for loop to iterate over the files and print their names
for folder in folders:
    if not folder.startswith("."):
        new_dir = directory+folder
        file_list = os.listdir(new_dir)
        # Check if the file name starts with a .
        for file in file_list:
            if not file.startswith("."):
                new_file = new_dir+'/'+file
                files += [new_file]

## Define base class for pickling

In [3]:
# Should we double check that the products from the mapped reaction and the molecules marked as 'product' are actually the same? 
# I wonder which mapped reaction has 4 products...
# What do we do if there's any discrepency?...
# We should trust the mapped reaction more
# I'll create a new column for the mapped products, and then in the cleaning, filter out any molecules that have been marked as a product
#   but don't actually appear in the mapped reaction

# Create a map of the unique molecules names with Pura, and apply that map to the df! Much 
#   more efficient than applying Pura to everything! 

# Let's just assume that we can trust the marked products, and add both the marked products and mapped products
#   to the final df

In [4]:
class OrdToPickle():
    """
    Read in an ord file, check if it contains USPTO data, and then:
    1) Extract all the relevant data (raw): reactants, reagents, products, yields, temp, time
    2) Use Pura to sanitise all the molecules. Do it on a list at a time, it's faster!
    3) Canonicalise all the molecules
    """

    def __init__(self, ord_file_path):
        self.ord_file_path = ord_file_path
        self.data = message_helpers.load_message(self.ord_file_path, dataset_pb2.Dataset)
        self.filename = self.data.name

    def find_smiles(self, identifiers):
        for i in identifiers:
            if i.type == 2:
                smiles = self.clean_smiles(i.value)
                return smiles
        for ii in identifiers: #if there's no smiles, return the name
            if ii.type == 6:
                name = ii.value
                self.includes_name = True
                return name
        return np.nan

    def clean_mapped_smiles(self, smiles):
        # remove mapping info and canonicalsie the smiles at the same time
        # converting to mol and back canonicalises the smiles string
        try:
            m = Chem.MolFromSmiles(smiles)
            for atom in m.GetAtoms():
                atom.SetAtomMapNum(0)
            cleaned_smiles = Chem.MolToSmiles(m)
            return cleaned_smiles
        except AttributeError:
            self.includes_name = True
            return smiles

    def clean_smiles(self, smiles):
        # remove mapping info and canonicalsie the smiles at the same time
        # converting to mol and back canonicalises the smiles string
        try:
            cleaned_smiles = Chem.CanonSmiles(smiles)
            return cleaned_smiles
        except:
            self.includes_name = True
            return smiles

    #its probably a lot faster to sanitise the whole thing at the end
    # NB: And create a hash map/dict



    def build_rxn_lists(self):
        mapped_rxn_all = []
        reactants_all = []
        reagents_all = []
        marked_products_all = []
        mapped_products_all = []
        products_all = []
        solvents_all = []
        catalysts_all = []

        temperature_all = []

        rxn_times_all = []

        not_mapped_products_all = []
        yields_all = []
        includes_name_all = []

        for i in range(len(self.data.reactions)):
            rxn = self.data.reactions[i]
            # handle rxn inputs: reactants, reagents etc
            reactants = []
            reagents = []
            solvents = []
            catalysts = []
            marked_products = []
            mapped_products = []
            products = []
            not_mapped_products = []
            
            temperatures = []

            rxn_times = []

            yields = []
            mapped_yields = []

            self.includes_name = False
            

            #if reaction has been mapped, get reactant and product from the mapped reaction
            #Actually, we should only extract data from reactions that have been mapped
            is_mapped = self.data.reactions[i].identifiers[0].is_mapped
            if is_mapped:
                mapped_rxn_extended_smiles = self.data.reactions[i].identifiers[0].value
                mapped_rxn = mapped_rxn_extended_smiles.split(' ')[0]

                reactant, reagent, mapped_product = mapped_rxn.split('>')

                for r in reactant.split('.'):
                    if '[' in r and ']' in r and ':' in r:
                        reactants += [r]
                    else:
                        reagents += [r]

                reagents += [r for r in reagent.split('.')]

                for p in mapped_product.split('.'):
                    if '[' in p and ']' in p and ':' in p:
                        mapped_products += [p]
                        
                    else:
                        not_mapped_products += [p]


                # inputs
                for key in rxn.inputs: #these are the keys in the 'dict' style data struct
                    try:
                        components = rxn.inputs[key].components
                        for component in components:
                            rxn_role = component.reaction_role #rxn role
                            identifiers = component.identifiers
                            smiles = self.find_smiles(identifiers)
                            if rxn_role == 1: #reactant
                                #reactants += [smiles]
                                # we already added reactants from mapped rxn
                                # So instead I'll add it to the reagents list
                                # A lot of the reagents seem to have been misclassified as reactants
                                # I just need to remember to remove reagents that already appear as reactants
                                #   when I do cleaning

                                reagents += [r for r in smiles.split('.')]
                            elif rxn_role ==2: #reagent
                                reagents += [r for r in smiles.split('.')]
                            elif rxn_role ==3: #solvent
                                solvents += [smiles]
                            elif rxn_role ==4: #catalyst
                                catalysts += [smiles]
                            elif rxn_role in [5,6,7]: #workup, internal standard, authentic standard. don't care about these
                                continue
                            # elif rxn_role ==8: #product
                            #     #products += [smiles]
                            # there are no products recorded in rxn_role == 8, they're all stored in "outcomes"
                    except IndexError:
                        #print(i, key )
                        continue

                # temperature
                try:
                    temperatures +=[rxn.conditions.temperature.control.type]
                except IndexError:
                    continue

                #rxn time
                try:
                    rxn_times = (rxn.outcomes[0].reaction_time.value, rxn.outcomes[0].reaction_time.units)
                except IndexError:
                    continue

                # products & yield
                products_obj = rxn.outcomes[0].products
                y1 = np.nan
                y2 = np.nan
                for marked_product in products_obj:
                    try:
                        identifiers = marked_product.identifiers
                        product_smiles = self.find_smiles(identifiers)
                        measurements = marked_product.measurements
                        for measurement in measurements:
                            if measurement.details =="PERCENTYIELD":
                                y1 = measurement.percentage.value
                            elif measurement.details =="CALCULATEDPERCENTYIELD":
                                y2 = measurement.percentage.value
                        #marked_products += [(product_smiles, y1, y2)]
                        marked_products += [product_smiles]
                        if y1 == y1:
                            yields += [y1]
                        elif y2==y2:
                            yields +=[y2]
                        else:
                            yields += [np.nan]
                    except IndexError:
                        continue
            
            #clean the smiles

            #remove reagents that are integers
            #reagents = [x for x in reagents if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]
            # I'm assuming there are no negative integers
            reagents = [x for x in reagents if not (x.isdigit())]

            reactants = [self.clean_mapped_smiles(smi) for smi in reactants]
            reagents = [self.clean_smiles(smi) for smi in reagents]
            solvents = [self.clean_smiles(smi) for smi in solvents]
            catalysts = [self.clean_smiles(smi) for smi in catalysts]

            # if the reagent exists in another list, remove it
            reagents_trimmed = []
            for reag in reagents:
                if reag not in reactants and reag not in solvents and reag not in catalysts:
                    reagents_trimmed += [reag]
            

            mapped_rxn_all += [mapped_rxn]
            reactants_all += [reactants]
            reagents_all += [list(set(reagents_trimmed))]
            solvents_all += [list(set(solvents))]
            catalysts_all += [list(set(catalysts))]
            
            temperature_all = [temperatures]

            rxn_times_all += [rxn_times]


            # products logic
            # handle the products
            # for each row, I will trust the mapped product more
            # loop over the mapped products, and if the mapped product exists in the marked product
            # add the yields, else simply add smiles and np.nan

            # canon and remove mapped info from products
            mapped_p_clean = [self.clean_mapped_smiles(p) for p in mapped_products]
            marked_p_clean = [self.clean_smiles(p) for p in marked_products]
            # What if there's a marked product that only has the correct name, but not the smiles?



            for mapped_p in mapped_p_clean:
                added = False
                for ii, marked_p in enumerate(marked_p_clean):
                    if mapped_p == marked_p and mapped_p not in products:
                        products+= [mapped_p]
                        mapped_yields += [yields[ii]]
                        added = True
                        break

                if not added and mapped_p not in products:
                    products+= [mapped_p]
                    mapped_yields += [np.nan]
            

            products_all += [products] 
            yields_all +=[mapped_yields]
            includes_name_all += [self.includes_name]


        
        return mapped_rxn_all, reactants_all, reagents_all, solvents_all, catalysts_all, temperature_all, rxn_times_all, products_all, yields_all, includes_name_all

    # create the column headers for the df
    def create_column_headers(self, df, base_string):
        column_headers = []
        for i in range(len(df.columns)):
            column_headers += [base_string+str(i)]
        return column_headers
    
    def build_full_df(self):
        headers = ['mapped_rxn_', 'reactant_', 'reagents_',  'solvent_', 'catalyst_', 'temperature_', 'rxn_time_', 'product_', 'yields_', 'includes_name_']
        #data_lists = [mapped_rxn, reactants_all, reagents_all, solvents_all, catalysts_all, temperature_all, rxn_times_all, products_all]
        data_lists = self.build_rxn_lists()
        for i in range(len(headers)):
            new_df = pd.DataFrame(data_lists[i])
            df_headers = self.create_column_headers(new_df, headers[i])
            new_df.columns = df_headers
            if i ==0:
                full_df = new_df
            else:
                full_df = pd.concat([full_df, new_df], axis=1)
        return full_df
    
    #def clean_df(self,df):
        # In the test case: data/ORD_USPTO/ord-data/data/59/ord_dataset-59f453c3a3d34a89bfd97b6b8b151908.pb.gz
        #   there is only 1 reaction with 9 catalysts, all other reactions have like 1 or 2
        #   Perhaps I should here add some hueristic that there's more than x unique catalysts, just filter out
        #   the whole reaction
        

    def main(self):
        # This function doesn't return anything. Instead, it saves the requested data as a pickle file at the path you see below
        # So you need to unpickle the data to see the output
        if 'uspto' in self.filename:
            full_df = self.build_full_df()
            #cleaned_df = self.clean_df(full_df)

            #save to pickle
            filename = self.data.name
            full_df.to_pickle(f"data/ORD_USPTO/pickled_data/{filename}.pkl")
        else:
            print(f'The following does not contain USPTO data: {self.filename}')



In [230]:
instance = OrdToPickle(files[0])
instance.main()

In [231]:
files[0]

'data/ORD_USPTO/ord-data/data/59/ord_dataset-59f453c3a3d34a89bfd97b6b8b151908.pb.gz'

In [232]:
#unpickle
filename = instance.filename
unpickled_df = pd.read_pickle(f"data/ORD_USPTO/pickled_data/{filename}.pkl")


In [233]:
unpickled_df.columns

Index(['mapped_rxn_0', 'reactant_0', 'reactant_1', 'reactant_2', 'reactant_3',
       'reactant_4', 'reagents_0', 'reagents_1', 'reagents_2', 'reagents_3',
       'reagents_4', 'reagents_5', 'reagents_6', 'reagents_7', 'reagents_8',
       'solvent_0', 'solvent_1', 'solvent_2', 'solvent_3', 'solvent_4',
       'solvent_5', 'catalyst_0', 'catalyst_1', 'catalyst_2', 'catalyst_3',
       'catalyst_4', 'catalyst_5', 'catalyst_6', 'catalyst_7', 'temperature_0',
       'rxn_time_0', 'rxn_time_1', 'product_0', 'product_1', 'product_2',
       'product_3', 'yields_0', 'yields_1', 'yields_2', 'yields_3',
       'includes_name_0'],
      dtype='object')

In [237]:
unpickled_df.yields_0.dropna()

0        58.299999
1        25.700001
2        92.800003
3        93.000000
4        98.000000
           ...    
1570     98.000000
1571     35.000000
1572    100.000000
1573     77.000000
1574     46.000000
Name: yields_0, Length: 781, dtype: float64

In [234]:
df = unpickled_df
df = df.loc[df.includes_name_0, :]

In [235]:
Chem.CanonSmiles('O')

'O'

In [236]:
df.iloc[0]

mapped_rxn_0       [CH3:1][S:2][C:3]1[O:7][C:6]([C:8]2[CH:13]=[CH...
reactant_0                           CSc1nnc(-c2ccccc2CS(N)(=O)=O)o1
reactant_1                           COc1cc(OC)nc(NC(=O)Oc2ccccc2)n1
reactant_2                                                      None
reactant_3                                                      None
reactant_4                                                      None
reagents_0                                                        Cl
reagents_1                                         1,5-diazbicyclo[5
reagents_2                                              CCCCC=CCCCCC
reagents_3                                                      None
reagents_4                                                      None
reagents_5                                                      None
reagents_6                                                      None
reagents_7                                                      None
reagents_8                        

In [184]:
unpickled_df.iloc[261]['mapped_rxn_0']

'[H-].[Na+].C(OC(C)(C)C)(=O)[CH2:4][C:5]([O:7]C(C)(C)C)=[O:6].[F:18][C:19]1[C:24]([C:25]([O:27][CH3:28])=[O:26])=[C:23]([F:29])[C:22]([F:30])=[C:21](F)[C:20]=1[F:32].Cl>CN(C)C=O.FC(F)(F)C(O)=O.C1(C)C=CC=CC=1.O.C(OCC)C.C(OCC)(=O)C>[C:5]([CH2:4][C:21]1[C:22]([F:30])=[C:23]([F:29])[C:24]([C:25]([O:27][CH3:28])=[O:26])=[C:19]([F:18])[C:20]=1[F:32])([OH:7])=[O:6]'

In [191]:
#pb = 'data/ORD_USPTO/ord-data/data/02/ord_dataset-02ee2261663048188cf6d85d2cc96e3f.pb.gz'
#data = message_helpers.load_message(pb, dataset_pb2.Dataset)
data = message_helpers.load_message(files[0], dataset_pb2.Dataset)

In [254]:
data.reactions[0].outcomes[0].products[0].measurements

[type: AMOUNT
details: "MASS"
amount {
  mass {
    value: 49.0
    units: GRAM
  }
}
, type: YIELD
details: "CALCULATEDPERCENTYIELD"
percentage {
  value: 58.3
}
]

In [5]:
# pickle everything
for file in tqdm(files):
    instance = OrdToPickle(file)
    instance.main()

  1%|          | 6/515 [01:21<1:42:31, 12.08s/it]

The following does not contain USPTO data: 


  3%|▎         | 18/515 [05:10<2:42:45, 19.65s/it]

The following does not contain USPTO data: 


  4%|▍         | 21/515 [07:16<3:51:00, 28.06s/it]

...

 87%|████████▋ | 446/515 [3:52:02<24:10, 21.02s/it]

The following does not contain USPTO data: Test data from https://doi.org/10.1039/C8SC04228D


 90%|█████████ | 464/515 [3:58:53<19:12, 22.59s/it]

The following does not contain USPTO data: Electroreductive Coupling of Alkenyl and Benzyl Halides


100%|██████████| 515/515 [4:20:29<00:00, 30.35s/it]


## Read in a complete df

## Use Pura

In [196]:
resolved = resolve_identifiers(
    ['benzene'],
    input_identifer_type=CompoundIdentifierType.NAME,
    output_identifier_type=CompoundIdentifierType.SMILES,
    services=[PubChem(autocomplete=True), CIR(), CAS()],
    agreement=1,
    silent=True,
)

pura_smiles = resolved[0][1][0]
pura_smiles

Batch: 100%|██████████| 1/1 [00:00<00:00,  1.53it/s]


'c1ccccc1'

## Read in the pickle

In [ ]:
import pandas as pd
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
import pandas as pd
from tqdm import tqdm

In [ ]:
#
#create one big df of all the pickled data
self.folder_path = 'data/ORD_USPTO/pickled_data/'
onlyfiles = [f for f in listdir(self.folder_path) if isfile(join(self.folder_path, f))]
full_df = pd.DataFrame()
for file in tqdm(onlyfiles):
    if file[0] != '.': #We don't want to try to unpickle .DS_Store
        filepath = self.folder_path+file 
        unpickled_df = pd.read_pickle(filepath)
        full_df = pd.concat([full_df, unpickled_df], ignore_index=True)


100%|██████████| 491/491 [05:35<00:00,  1.46it/s]


In [ ]:
full_df

NameError: name 'full_df' is not defined

In [ ]:
class USPTO_cleaning():
    def __init__(self, folder_path) -> None:
        self.folder_path = folder_path
    
    def create_full_df(self):
        #create one big df of all the pickled data
        #folder_path = 'data/ORD_USPTO/pickled_data/'
        onlyfiles = [f for f in listdir(self.folder_path) if isfile(join(self.folder_path, f))]
        full_df = pd.DataFrame()
        for file in tqdm(onlyfiles):
            if file[0] != '.': #We don't want to try to unpickle .DS_Store
                filepath = self.folder_path+file 
                unpickled_df = pd.read_pickle(filepath)
                full_df = pd.concat([full_df, unpickled_df], ignore_index=True)
        return full_df

    def clean_data(self, df):
        # 
        



In [ ]:
folder_path = 'data/ORD_USPTO/pickled_data/'
full_USPTO_data_df = USPTO_cleaning(folder_path)


# Preprocessing of USPTO - Molecular AI

In [ ]:
# Running code from:
# https://molecularai.github.io/reaction_utils/uspto.html

# From within the folder containing all the USPTO data

# First run:
# conda activate <rxnutilities>
#  python -m rxnutils.data.uspto.preparation_pipeline run --nbatches 200  --max-workers 8 --max-num-splits 200

# Then I was supposed to run:
# conda activate rxnmapper
# python -m rxnutils.data.mapping_pipeline run --data-prefix uspto --nbatches 200  --max-workers 8 --max-num-splits 200
# But that didn't work, so I just ran the first part 
# Even after I replaced the delimiter (from 	 to , it still failed ). I'll just give up lol

## Read in data cleaned by rxn utils

In [ ]:
import pandas as pd

In [ ]:
cleaned_USPTO = pd.read_csv('/Users/dsw46/USPTO_data/uspto_data_cleaned.csv', sep = '	')
cleaned_USPTO.shape

(3740596, 7)

In [ ]:
cleaned_USPTO['ReactionSmilesClean'][0]

'OCCBr.CCS(=O)(=O)Cl.CCOCC.CCN(CC)CC>>CCS(=O)(=O)OCCBr'

In [ ]:
#full_USPTO = pd.read_csv('/Users/dsw46/USPTO_data/uspto_data.csv', sep = '	')
#full_USPTO.shape